In [1]:
!pip install tweepy

     |████████████████████████████████| 147 kB 3.8 MB/s eta 0:00:01
You should consider upgrading via the '/Users/christopherkindl/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
import tweepy
import numpy as np
import pandas as pd

In [2]:
consumer_key = 'DlidHq2CnP45yMJIhrX8pdKNc'
consumer_secret = 'loXSV3wx5GgtsQL4NXzyhF05w3JEsMuNo0OkYzFxxf6HmiKHym'
access_token = '1384290357542719489-DcyoxpaVdgt3o7epbAYR4apUokvqng'
access_token_secret = '2Irv2RJSjgTgvTuRsSYnzDYlfqREFKF8OK8YtCRYTqySP'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAO7tOgEAAAAAg98XSCXlF6HtnXPLueDo%2BhSngbs%3DhfuVfM9kDdNNGZxwRmFz06I9dAvZ72nhKlnXTglnVbnsYpGatN'

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [15]:
stations = pd.read_csv ('/Users/christopherkindl/working/data-engineering-group-assignment/\
01_web_scraping/Transport Data.csv')

In [17]:
stations

,Station,OS X,OS Y,Latitude,Longitude,Zone,Postcode
0,Abbey Road,539081,183352,51.531952,0.003723,3,E15 3NB
1,Abbey Wood,547297,179002,51.490784,0.120272,4,SE2 9RH
2,Acton Central,520613,180299,51.508758,-0.263430,2,W3 6BH
3,Acton Main Line,520296,181196,51.516887,-0.267690,3,W3 9EH
4,Acton Town,519457,179639,51.503071,-0.280303,3,W3 8HN
...,...,...,...,...,...,...,...
647,Woodside Park,525725,192564,51.617868,-0.185426,4,N12 8SE
648,Woolwich,543931,178994,51.491578,0.071819,4,NaN
649,Woolwich Arsenal,543754,178803,51.489907,0.069194,4,SE18 6HX
650,Woolwich Dockyard,542738,178908,51.491108,0.054612,3,SE18 5JY


In [22]:
# scrape tweets from a particular station with a radius of 1km
geocode= str(stations['Latitude'][index])+','+str(stations['Longitude'][index])+', 2km'

for index in range(len(stations[:3])):
    print(str(stations['Latitude'][index])+','+str(stations['Longitude'][index])+',2km')

51.53195199,0.0037233709999999996, 2km
51.4907841,0.12027197, 2km
51.50875778,-0.263430199, 2km


In [33]:
number_of_tweets = 5
tweets = []
station = []
date = []

for index in range(len(stations[:3])):
    print('about to scrape tweets from station ', stations['Station'][index])
    for i in tweepy.Cursor(api.search, q = 'london', lang = 'en', geocode= \
                           str(stations['Latitude'][index])+','+str(stations['Longitude'][index])+',1km').\
        items(number_of_tweets):
        tweets.append(i.text)
        station.append(stations['Station'][index])
        date.append(i.created_at)

about to scrape tweets from station  Abbey Road
about to scrape tweets from station  Abbey Wood
about to scrape tweets from station  Acton Central


In [34]:
df = pd.DataFrame({'tweets': tweets, 'date':date, 'station': station})

In [35]:
df

,tweets,date,station
0,While there is massive upheaval going on in th...,2021-04-19 11:26:43,Abbey Road
1,"""Craft paper/card"" in #London #UnitedKingdom h...",2021-04-13 09:18:11,Abbey Road
2,"""Tesco White baguette"" in #London #UnitedKingd...",2021-04-13 09:04:44,Abbey Road
3,"""Large dashboard monthly happy planner Jan 202...",2021-04-13 03:18:11,Abbey Road
4,"""pink mirror and watch "" in #London #UnitedKin...",2021-04-12 21:18:11,Abbey Road
5,"Just posted a photo @ Abbey Wood, London, SE2 ...",2021-04-20 07:00:25,Abbey Wood
6,"Just posted a video @ Bedford Park, London htt...",2021-04-19 17:03:53,Acton Central
7,Someone please transport me to the future... @...,2021-04-13 16:39:05,Acton Central
8,"""Door numbers wanted"" in #London #UnitedKingdo...",2021-04-13 03:05:10,Acton Central
